In [1]:
import torch
torch.cuda.is_available()

False

In [1]:
# %% test a specific model
from src.module import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-04-30 05:46:45|MOD:display     |: Model Specifics:
24-04-30 05:46:45|MOD:display     |: Start Process [Data] at Tue Apr 30 05:46:45 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_day_ShortTest.1!
Callback : DynamicDataLink() , assign/unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , rest optimizer on certain epoch (speedup scheduler may apply)
Callback : ProcessTimer() , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr when fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain when fitting encounters nan loss
Callback : LoaderDisplay(batch_interval=1) , display batch progress bar
Callback : ProgressDisplay(verbosity=10) , display epoch/event information
{'random_seed': None,
 'model_name': 'gru_day_ShortTest.1',
 'model_mod

24-04-30 05:46:46|MOD:display     |: Finish Process [Data], Cost 1.8 Secs
24-04-30 05:46:46|MOD:display     |: Start Process [Fit] at Tue Apr 30 05:46:46 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [2]:
1 <= 5 <= 6

True

In [ ]:
# prepare data
from src.module import DataModule
DataModule.prepare_data()

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds


In [2]:
import inspect

def my_function(a, b):
    pass

# Assert that the function has exactly two arguments with names 'a' and 'b'
assert len(inspect.signature(my_function).parameters) == 2
assert inspect.signature(my_function).parameters['a'].name == 'a'
assert inspect.signature(my_function).parameters['b'].name == 'b'

print("Function 'my_function' has two arguments 'a' and 'b'")


Function 'my_function' has two arguments 'a' and 'b'


In [18]:
[v.name for v in inspect.signature(my_function).parameters.values()]

['a', 'b']